# Plynty Bureau of Labor Statistics Consumer Expenditure Analysis

[BLS Comsumer Expenditure Survey](https://www.bls.gov/cex/home.htm)

[Interview Data Dictionary](https://www.bls.gov/cex/2015/csxintvwdata.pdf)

[Diary Data Dictionary](https://www.bls.gov/cex/2015/csxdiarydata.pdf)

### Where to download the BLS CE PUMD
- The zip files download automatically
- To download the Stub files open the links then right click and choose "Save As..."

[2015 interview zip file](https://www.bls.gov/cex/pumd/data/comma/intrvw15.zip)

[2015 diary zip file](https://www.bls.gov/cex/pumd/data/comma/diary15.zip)

[2015 IntStub file](https://www.bls.gov/cex/pumd/2014/csxintstub.txt)

[2015 IStub file](https://www.bls.gov/cex/pumd/2014/csxistub.txt)

[2015 DStub file](https://www.bls.gov/cex/pumd/2014/csxdstub.txt)

### This Scripts Goals for Plynty
- Create an easy to use analysis script for the BLS CE PUMD
- Create a csv files that has average percentages spent on plynty categories for certain income classes
- Create incomeclasses that are significant

##### Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import math
import matplotlib.pyplot as plt
from scipy.interpolate import *
from plyntywidgets import *
from blsFunctions import *
print("Dependencies Loaded...")

Dependencies Loaded...


### Setting Parameters
- year: the last two number associated with the year of the data
    for example for data from 2015: year = "15"
- minAge: the low bound (inclusive) of the age range you wish to subset by
- maxAge: the high bound (inclusive) of the age range you wish to subset by
- incomeBrackets: array of numbers that you wish to create the new income classes
    the bracketing works as follows (1,2], (2,3], (3,4]
- filesToRead: the strings of the abbreviations associated with the files you wish to read
    options are: "all", "diary", "interview", "dtbd", "expd", "fmld", "memd", "fmli", "itbi", "memi", "mtbi", "ntaxi"

In [2]:
year = "15"
filesToRead = ["fmli", "mtbi"]
incomeBrackets = [-10000000, 10000, 30000, 50000, 60000, 70000, 80000, 100000, 150000, 9990000]
familyIncomeBrackets = incomeBrackets
singleIncomeBrackets = incomeBrackets


# minAge = 55
# maxAge = 64

minAge = 60
maxAge = 75

# minAge = 65
# maxAge = 130

### Setting Directory locations and FileNames on your Local Machine

In [3]:
# directory in which the diary and interview folders are held is located
diaryDir = "/Users/adyke/Vizuri/CE_PUMD/diary15/"
interviewDir = "/Users/adyke/Vizuri/CE_PUMD/intrvw15/"

# Directory where stubfiles are located
pathToStubFileDir = "/Users/adyke/Vizuri/Stubfiles/"
rScriptStubfilePathAndName = "/Users/adyke/Vizuri/BLS_Python_Analysis/creatingStubCsvs.R"

# Filenames of the Stubfiles
IStubFileName = "IStub2015.txt"
DStubFileName = "DStub2015.txt"
IntStubFileName = "IntStub2015.txt"

# name of interview dir within the interview dir
insideIntrvwDirName = "intrvw"

# name of the directory where you want the output percentages csv
outputDir = "/Users/adyke/Vizuri/outputFiles/"

### Reading in the files specified by FilesToRead

In [4]:
if(len(filesToRead)==0):
    print("The files to read variable is empty.")

# looping through each file to read
for file in filesToRead:
    if file == "dtbd" or file == "all" or file == "diary":
        dtbd = readFileSet("dtbd", diaryDir)
    if file == "expd" or file == "all" or file == "diary":
        expd = readFileSet("expd", diaryDir)
    if file == "fmld" or file == "all" or file == "diary":
        fmld = readFileSet("fmld", diaryDir)
    if file == "memd" or file == "all" or file == "diary":
        memd = readFileSet("memd", diaryDir)
    if file == "fmli" or file == "all" or file == "interview":
        fmli = readFileSet("fmli", interviewDir+insideIntrvwDirName+year+"/")
    if file == "itbi" or file == "all" or file == "interview":
        itbi = readFileSet("itbi", interviewDir+insideIntrvwDirName+year+"/")
    if file == "itii" or file == "all" or file == "interview":
        itii = readFileSet("itii", interviewDir+insideIntrvwDirName+year+"/")
    if file == "memi" or file == "all" or file == "interview":
        memi = readFileSet("memi", interviewDir+insideIntrvwDirName+year+"/")
    if file == "mtbi" or file == "all" or file == "interview":
        mtbi = readFileSet("mtbi", interviewDir+insideIntrvwDirName+year+"/")
        mtbi.UCC = mtbi.UCC.astype(str)
    if file == "ntaxi" or file == "all" or file == "interview":
        ntaxi = readFileSet("ntaxi", interviewDir+insideIntrvwDirName+year+"/")
# does not read form the expn or para subdirectories


### Using R to convert the Stub files into csv files

In [5]:
if os.path.isfile(pathToStubFileDir+"DStub.csv") and os.path.isfile(pathToStubFileDir+"IStub.csv") and os.path.isfile(pathToStubFileDir+"IntStub.csv"):
    print("Stubfiles Exist")
else:
    # converting the stub files via R 
    subprocess.call("Rscript "+rScriptStubfilePathAndName+" "+pathToStubFileDir+" "+IStubFileName+" "+DStubFileName+" "+IntStubFileName, shell=True)
    print("Stubfile Csvs created in "+pathToStubFileDir)

Stubfiles Exist


### Reading and Cleaning the stubfile CSVs into pandas dataframes

In [6]:
# reading in the stubfiles
DStub = pd.read_csv(pathToStubFileDir+"DStub.csv")
IStub = pd.read_csv(pathToStubFileDir+"IStub.csv")
IntStub = pd.read_csv(pathToStubFileDir+"IntStub.csv")

# removing the index from the stufile
DStub = DStub.drop(DStub.columns[0], axis=1)
IStub = IStub.drop(IStub.columns[0], axis=1)
IntStub = IntStub.drop(IntStub.columns[0], axis=1)

# replacing * with 0 in the level columns
DStub.loc[DStub.level == "*", 'level'] = 0
IStub.loc[IStub.level == "*", 'level'] = 0
IntStub.loc[IntStub.level == "*", 'level'] = 0

# Starting the Plynty calculations

### Creating the UCC roll ups for Plynty
##### What is a UCC?
- Universial Classification Code
- Used to classify different expenditure sub categories

### Adding and Rolling up the MTBI Categories into mtbiRolledUp
##### What is the MTBI file?
- Monthly expenditure file classified by UCC
- Used to populate expenditures in each category for each CU

In [7]:
monthlyHousing = ["220311","220313","880110","210110","800710","210901","220312","220314","880310","210902"]
monthlyHousing.extend(categoricalUCCRollUp(IStub, ["UTILS"]))

rollupDict = {"iTotalExp":(categoricalUCCRollUp(IStub,["TOTALE"])),
"iFoodAtHome":(categoricalUCCRollUp(IStub, ["FOODHO", "ALCHOM"])),
"iFoodAway":(categoricalUCCRollUp(IStub, ["FOODAW", "ALCAWA"])),
"iHousing":(["220311","220313","880110","210110","800710","210901","220312","220314","880310","210902"]),
"otherHousing":(categoricalUCCRollUp(IStub, ["HOUSIN"], ignoreUCCs = monthlyHousing)),
"iUtilites":(categoricalUCCRollUp(IStub, ["UTILS"])),
"iClothingAndBeauty":(categoricalUCCRollUp(IStub, ["APPARE","PERSCA"])),
"iTransportation":(categoricalUCCRollUp(IStub, ["TRANS"])),
"iHealthcare":(categoricalUCCRollUp(IStub, ["HEALTH"])),
"iEntertainment":(categoricalUCCRollUp(IStub, ["ENTRTA","READIN"])),
"iMiscellaneous":(categoricalUCCRollUp(IStub, ["MISC","TOBACC"])),
"iCharitableAndFamilyGiving":(categoricalUCCRollUp(IStub, ["CASHCO"])),
"iInsurance":(categoricalUCCRollUp(IStub, ["LIFEIN"])),
"iEducation":(categoricalUCCRollUp(IStub, ["EDUCAT"])),
"iHousingPrinciple":(categoricalUCCRollUp(IStub,["MRTPRI"]))}

# there is a multiple of 4 because each survey lasts for 3 months 
# multiplying by 4 gives a estimate for each year (3 * 4 = 12)
mtbiRolledUp = rollUpDataframeDict(mtbi, rollupDict, negativeColumns=["iHousingPrinciple"], multiple=4)

mtbiTrimmed = mtbiRolledUp.loc[: , ['NEWID','iTotalExp','iFoodAtHome','iFoodAway','iHousing','otherHousing','iUtilites','iClothingAndBeauty','iTransportation','iHealthcare','iEntertainment','iMiscellaneous','iCharitableAndFamilyGiving','iInsurance','iEducation','iHousingPrinciple']]

### Creating the Sum for all expenditure category columns for each NEWID
- nonZeroColumns is an array that contains the names of columns that should not be Zero

outputs from cell:
- iExpensesByNewID: total expenses for each category for each NewID

In [8]:
# adding up all columns for each new id
iExpensesByNewID = mtbiTrimmed.groupby(['NEWID'],as_index=False).sum()
# removing rows with zero values in key categories
nonZeroColumns = ['iFoodAtHome','iFoodAway','iHousing','iUtilites']
for column in nonZeroColumns:
    iExpensesByNewID = iExpensesByNewID[iExpensesByNewID[column] != 0]
iExpensesByNewID['iHousing'] = iExpensesByNewID['iHousing']+iExpensesByNewID['iHousingPrinciple']
iExpensesByNewID['iTotalExp'] += iExpensesByNewID['iHousingPrinciple']

### Subsetting FMLI for age and recoding the incomebrackets
##### What is the fmli file?
- File with entries for each CU, characteristics, and summary level expenditures

In [9]:
# subsetting for the age bracket
fmliAge = subsetDataframe(dataframe=fmli, columnName="AGE_REF", minValue=minAge, maxValue=maxAge)
fmliAge = fmliAge.reset_index()

fmliFamilyAge = subsetDataframe(dataframe=fmliAge, columnName="FAM_SIZE", minValue = 2, maxValue = 100)
fmliSingleAge = subsetDataframe(dataframe=fmliAge, columnName="FAM_SIZE", minValue = 1)

In [10]:
fmliFamilyRecoded = binColumn(dataframe=fmliFamilyAge, toBinColumnName="FINCBTXM", binValues=familyIncomeBrackets, binnedColumnName="INCLASS", labels=range(1,len(familyIncomeBrackets)))
inclassExpensesFamily = pd.merge(left=fmliFamilyRecoded[['NEWID','INCLASS','FINCBTXM']],right=iExpensesByNewID, on=['NEWID'])
print(inclassExpensesFamily)
inclassAveragesFamily = round(inclassExpensesFamily.ix[: ,inclassExpensesFamily.columns != 'NEWID'].groupby(['INCLASS'],as_index=False).mean(),2)
percentagesFamily = inclassAveragesFamily.loc[:,rollupDict.keys()]
for column in rollupDict.keys():
    percentagesFamily[column] = inclassAveragesFamily[column]/inclassAveragesFamily.iTotalExp
percentagesFamily['ExpInc'] = inclassAveragesFamily['iTotalExp']/inclassAveragesFamily['FINCBTXM']
percentagesFamily.ExpInc.ix[percentagesFamily['ExpInc']>1] = 1
outliersFamily = inclassExpensesFamily.copy()
outerFenceFamily = []
for column in outliersFamily.columns[4:len(outliersFamily.columns)-1]:
    Q1 = outliersFamily[column].quantile(0.25)
    Q3 = outliersFamily[column].quantile(0.75)
    IQR = Q3 - Q1
    outerFenceFamily.extend(outliersFamily[outliersFamily[column] < (Q1 - (3 * IQR))].index.tolist())
    outerFenceFamily.extend(outliersFamily[outliersFamily[column] > (Q3 + (3 * IQR))].index.tolist())
cleanFamily = outliersFamily.drop(outliersFamily.index[outerFenceFamily])
inclassCleanAveragesFamily = round(cleanFamily.ix[: ,cleanFamily.columns != 'NEWID'].groupby(['INCLASS'],as_index=False).mean(),2)
# creating new dataframe for the percentages that only includes the plynty categories
cleanPercentagesFamily = inclassCleanAveragesFamily.loc[:,rollupDict.keys()]
for column in rollupDict.keys():
    cleanPercentagesFamily[column] = inclassCleanAveragesFamily[column]/inclassCleanAveragesFamily.iTotalExp
cleanPercentagesFamily['ExpInc'] = inclassCleanAveragesFamily['iTotalExp']/inclassCleanAveragesFamily['FINCBTXM']
# truncate the max ExpInc
cleanPercentagesNonTruncatedFamily = cleanPercentagesFamily.copy()
cleanPercentagesFamily.ExpInc.ix[cleanPercentagesFamily['ExpInc']>1] = 1

        NEWID  INCLASS  FINCBTXM  iTotalExp  iFoodAtHome  iFoodAway  iHousing  \
0     2792295        5   62979.0    51170.8       7800.0     2600.0   11020.0   
1     2792645        5   63254.0   238725.6       6540.0     3132.0   12492.0   
2     2792825        9  211240.8   101121.0       3200.0     2360.0   16800.0   
3     2795165        7   82014.0   182693.6       5200.0     4420.0    6156.0   
4     2796055        8  145100.0    41987.8       4060.0      520.0     960.0   
5     2796245        8  116500.0    57564.0       5200.0     3120.0    8700.0   
6     2796625        3   42000.0    53940.0       8440.0     1040.0   11220.0   
7     2796735        2   26506.0    30822.8       4980.0     1040.0    9900.0   
8     2796765        3   43067.0    36526.8       1300.0      312.0    1488.0   
9     2797585        2   28954.4    43793.6       3240.0     1560.0    2376.0   
10    2797675        9  206724.2    64146.8       6440.0     1328.0   14400.0   
11    2798665        2   158

/Users/adyke/Vizuri/BLS_Python_Analysis/blsFunctions.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe[binnedColumnName] = pd.cut(dataframe[toBinColumnName], bins=binValues, labels=labels)


In [11]:
fmliSingleRecoded = binColumn(dataframe=fmliSingleAge, toBinColumnName="FINCBTXM", binValues=singleIncomeBrackets, binnedColumnName="INCLASS", labels=range(1,len(singleIncomeBrackets)))
inclassExpensesSingle = pd.merge(left=fmliSingleRecoded[['NEWID','INCLASS','FINCBTXM']],right=iExpensesByNewID, on=['NEWID'])
inclassAveragesSingle = round(inclassExpensesSingle.ix[: ,inclassExpensesSingle.columns != 'NEWID'].groupby(['INCLASS'],as_index=False).mean(),2)
percentagesSingle = inclassAveragesSingle.loc[:,rollupDict.keys()]
for column in rollupDict.keys():
    percentagesSingle[column] = inclassAveragesSingle[column]/inclassAveragesSingle.iTotalExp
percentagesSingle['ExpInc'] = inclassAveragesSingle['iTotalExp']/inclassAveragesSingle['FINCBTXM']
percentagesSingle.ExpInc.ix[percentagesSingle['ExpInc']>1] = 1
outliersSingle = inclassExpensesSingle.copy()
outerFenceSingle = []
for column in outliersSingle.columns[4:len(outliersSingle.columns)-1]:
    Q1 = outliersSingle[column].quantile(0.25)
    Q3 = outliersSingle[column].quantile(0.75)
    IQR = Q3 - Q1
    outerFenceSingle.extend(outliersSingle[outliersSingle[column] < (Q1 - (3 * IQR))].index.tolist())
    outerFenceSingle.extend(outliersSingle[outliersSingle[column] > (Q3 + (3 * IQR))].index.tolist())
cleanSingle = outliersSingle.drop(outliersSingle.index[outerFenceSingle])
inclassCleanAveragesSingle = round(cleanSingle.ix[: ,cleanSingle.columns != 'NEWID'].groupby(['INCLASS'],as_index=False).mean(),2)
# creating new dataframe for the percentages that only includes the plynty categories
cleanPercentagesSingle = inclassCleanAveragesSingle.loc[:,rollupDict.keys()]
for column in rollupDict.keys():
    cleanPercentagesSingle[column] = inclassCleanAveragesSingle[column]/inclassCleanAveragesSingle.iTotalExp
cleanPercentagesSingle['ExpInc'] = inclassCleanAveragesSingle['iTotalExp']/inclassCleanAveragesSingle['FINCBTXM']
# truncate the max ExpInc
cleanPercentagesNonTruncatedSingle = cleanPercentagesSingle.copy()
cleanPercentagesSingle.ExpInc.ix[cleanPercentagesSingle['ExpInc']>1] = 1

/Users/adyke/Vizuri/BLS_Python_Analysis/blsFunctions.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe[binnedColumnName] = pd.cut(dataframe[toBinColumnName], bins=binValues, labels=labels)


In [12]:
cleanPercentagesSingle

,iTotalExp,iFoodAtHome,iFoodAway,iHousing,otherHousing,iUtilites,iClothingAndBeauty,iTransportation,iHealthcare,iEntertainment,iMiscellaneous,iCharitableAndFamilyGiving,iInsurance,iEducation,iHousingPrinciple,ExpInc
0,1.0,0.157198,0.072090,0.308538,0.088447,0.097987,0.029561,0.102380,0.082963,0.038593,0.009679,0.012565,0.0,0.0,0.051634,1.000000
1,1.0,0.145997,0.060421,0.320281,0.082924,0.100973,0.022560,0.091800,0.099484,0.047385,0.009759,0.018415,0.0,0.0,0.042022,1.000000
2,1.0,0.115336,0.054450,0.309978,0.097011,0.090336,0.022526,0.109667,0.124713,0.045144,0.007469,0.023370,0.0,0.0,0.061888,0.776985
3,1.0,0.110730,0.053688,0.309394,0.134732,0.079767,0.024348,0.117754,0.092647,0.062035,0.003050,0.011854,0.0,0.0,0.088332,0.623910
4,1.0,0.102696,0.067483,0.342504,0.140093,0.109659,0.021394,0.079025,0.065158,0.048089,0.006525,0.017374,0.0,0.0,0.060829,0.530938
5,1.0,0.097972,0.073634,0.320493,0.132546,0.093261,0.021687,0.114004,0.074430,0.049835,0.002409,0.019728,0.0,0.0,0.062253,0.429707
6,1.0,0.095020,0.080923,0.315176,0.093782,0.078530,0.045657,0.112236,0.106409,0.042772,0.004609,0.024887,0.0,0.0,0.082787,0.424833
7,1.0,0.077687,0.074455,0.317558,0.144078,0.084362,0.027439,0.099291,0.103164,0.050193,0.004474,0.017299,0.0,0.0,0.108244,0.337080
8,1.0,0.055579,0.085258,0.368749,0.128244,0.069056,0.045661,0.105879,0.050964,0.077084,0.002111,0.011415,0.0,0.0,0.156475,0.210954


In [13]:
cleanPercentagesFamily

,iTotalExp,iFoodAtHome,iFoodAway,iHousing,otherHousing,iUtilites,iClothingAndBeauty,iTransportation,iHealthcare,iEntertainment,iMiscellaneous,iCharitableAndFamilyGiving,iInsurance,iEducation,iHousingPrinciple,ExpInc
0,1.0,0.143982,0.051760,0.239631,0.137913,0.100992,0.021448,0.102635,0.122152,0.046759,0.008464,0.022193,0.002070,0.0,0.074093,1.000000
1,1.0,0.149723,0.058038,0.241721,0.104785,0.105938,0.025751,0.104198,0.121105,0.050503,0.010035,0.023022,0.005183,0.0,0.061673,1.000000
2,1.0,0.133246,0.056037,0.241773,0.119990,0.102830,0.023154,0.111385,0.123542,0.043856,0.014043,0.024477,0.005668,0.0,0.070234,0.974456
3,1.0,0.133057,0.054645,0.215030,0.130447,0.104668,0.027224,0.122241,0.126817,0.047103,0.013748,0.019621,0.005399,0.0,0.068321,0.759277
4,1.0,0.124630,0.060734,0.206044,0.143875,0.098088,0.028241,0.128367,0.120181,0.048696,0.008797,0.028155,0.004192,0.0,0.066135,0.704116
5,1.0,0.132708,0.060125,0.214994,0.151092,0.087205,0.028218,0.106962,0.123831,0.053823,0.010503,0.024478,0.006061,0.0,0.063211,0.694756
6,1.0,0.121390,0.053041,0.202784,0.158540,0.097044,0.026956,0.110181,0.133563,0.050734,0.012024,0.027921,0.005822,0.0,0.081345,0.594793
7,1.0,0.116824,0.065721,0.219731,0.167594,0.098056,0.031637,0.113029,0.107325,0.042680,0.007112,0.025732,0.004559,0.0,0.092330,0.484401
8,1.0,0.110140,0.066165,0.248875,0.180252,0.087449,0.031623,0.124336,0.074577,0.041903,0.007775,0.023777,0.003128,0.0,0.078288,0.339772


In [14]:
cleanPercentagesFamily.to_csv("cleanPercentFamily1.csv",index=False, encoding='utf-8')
cleanPercentagesSingle.to_csv("cleanPercentSingle1.csv",index=False, encoding='utf-8')

In [15]:
len(fmli.CUID.unique())

14979

### Adding the Income class colum to the ExpensesByNewID dataframe

In [16]:
# combining the fmli and iExpensesByNewID
inclassExpenses = pd.merge(left=fmliRecoded[['NEWID','INCLASS','FINCBTXM']],right=iExpensesByNewID, on=['NEWID'])
# inclassExpenses

NameError: name 'fmliRecoded' is not defined

### Averaging the expenditures based on incomebrackets
inclassAverages is average money spent for each incomeclass

In [ ]:
# getting mean for all columns with the same income class besides newId and creating new dataframe
inclassAverages = round(inclassExpenses.ix[: ,inclassExpenses.columns != 'NEWID'].groupby(['INCLASS'],as_index=False).mean(),2)
# inclassAverages

### Converting the Average expenditures for income classes into percentages of expenditures
percentages is the percent of total expenditure for each category for each incomeclass 

In [ ]:
# creating new dataframe for the percentages that only includes the plynty categories
percentages = inclassAverages.loc[:,rollupDict.keys()]
for column in rollupDict.keys():
    percentages[column] = inclassAverages[column]/inclassAverages.iTotalExp
percentages['ExpInc'] = inclassAverages['iTotalExp']/inclassAverages['FINCBTXM']

# truncate the max ExpInc
percentages.ExpInc.ix[percentages['ExpInc']>1] = 1

# percentages

# Exploring Issues in the data

### Getting Description of the inclassExpenses dataframe
Description includes:
- count
- mean
- standard deviation
- min
- max
- quartiles

In [ ]:
# max and min of housing per income class
rowDescription = []
for inclass in range(1,len(incomeBrackets)):
    rowDescription.append(inclassExpenses.iHousing.loc[inclassExpenses.INCLASS == inclass].describe())
descriptions = pd.concat(rowDescription, axis=1)
descriptions.columns = range(0,len(incomeBrackets)-1)
descriptions = descriptions.transpose()
# descriptions

### Removing outliers in iHousing
 - For each income class
    - Find Q1 and Q3
    - Calculate IQR
    - Find rows outside of 
     - Inner Fence: Q3/Q2 +/- (1.5 x IQR)
     - Outer Fence: Q3/Q2 +/- (3 x IQR)
 - Remove Rows from Dataframe
 - Clean1: Inner Fence
 - Clean2: Outer Fence

In [ ]:
# cleaning the inclassExpenses dataframe of outliers
outliers1 = inclassExpenses.copy()


outliers1
innerFence = []
outerFence = []


print("There are "+str(len(inclassExpenses))+" rows before removal of outliers")

for inclass in range(1,len(incomeBrackets)):
    outliers1InClass = outliers1.where(outliers1['INCLASS']==inclass)
    Q1 = outliers1InClass['iHousing'].quantile(0.25)
    Q3 = outliers1InClass['iHousing'].quantile(0.75)
    IQR = Q3 - Q1
    innerFence.extend(outliers1InClass[outliers1InClass['iHousing'] < (Q1 - (1.5 * IQR))].index.tolist())
    innerFence.extend(outliers1InClass[outliers1InClass['iHousing'] > (Q3 + (1.5 * IQR))].index.tolist())    
    outerFence.extend(outliers1InClass[outliers1InClass['iHousing'] < (Q1 - (3 * IQR))].index.tolist())
    outerFence.extend(outliers1InClass[outliers1InClass['iHousing'] > (Q3 + (3 * IQR))].index.tolist())
#     Removing transportation outliers
#     Q1 = outliers1InClass['iTransportation'].quantile(0.25)
#     Q3 = outliers1InClass['iTransportation'].quantile(0.75)
#     IQR = Q3 - Q1
#     innerFence.extend(outliers1InClass[outliers1InClass['iTransportation'] < (Q1 - (1.5 * IQR))].index.tolist())
#     innerFence.extend(outliers1InClass[outliers1InClass['iTransportation'] > (Q3 + (1.5 * IQR))].index.tolist())    
#     outerFence.extend(outliers1InClass[outliers1InClass['iTransportation'] < (Q1 - (3 * IQR))].index.tolist())
#     outerFence.extend(outliers1InClass[outliers1InClass['iTransportation'] > (Q3 + (3 * IQR))].index.tolist())


clean1 = outliers1.drop(outliers1.index[innerFence])
clean1
clean2 = outliers1.drop(outliers1.index[outerFence])
clean2

print("Removed "+str(len(innerFence))+ " rows deemed to be out of inner fence")
print("Removed "+str(round(len(innerFence)/len(inclassExpenses)*100,2))+ "% of the CUs")

print("Removed "+str(len(outerFence))+ " rows deemed to be out of outer fence")
print("Removed "+str(round(len(outerFence)/len(inclassExpenses)*100,2))+ "% of the CUs")

# creating the descriptions for the cleaned outlier dataframe
rowDescription1 = []
for inclass in range(1,len(incomeBrackets)):
    rowDescription1.append(clean1.iHousing.loc[clean1.INCLASS == inclass].describe())
descriptions1 = pd.concat(rowDescription1, axis=1)
descriptions1.columns = range(0,len(incomeBrackets)-1)
descriptions1 = descriptions1.transpose()
# descriptions1

### Looking at the outliers that were removed

In [ ]:
innerOutliers = inclassExpenses.copy()
innerOutliers = innerOutliers.ix[innerFence,:]
print("Number of outliers in each income class: "+str(innerOutliers.INCLASS.value_counts().values))
# innerOutliers.head()

In [ ]:
outerOutliers = inclassExpenses.copy()
outerOutliers = outerOutliers.ix[outerFence,:]
print("Number of outliers in each income class: "+str(outerOutliers.INCLASS.value_counts().values))
# outerOutliers.head()

### Creating the percentage output for cleaned dataframe
##### What is the percentage dataframe?
- Dataframe where each row is a different income class and each column is a different plynty expenditure category
- These are percents of total expenditure (not of total income)

In [ ]:
# creating percentage outputs for cleaned dataframe
inclassCleanAverages1 = round(clean1.ix[: ,clean1.columns != 'NEWID'].groupby(['INCLASS'],as_index=False).mean(),2)
# creating new dataframe for the percentages that only includes the plynty categories
cleanPercentages1 = inclassCleanAverages1.loc[:,rollupDict.keys()]
for column in rollupDict.keys():
    cleanPercentages1[column] = inclassCleanAverages1[column]/inclassCleanAverages1.iTotalExp
cleanPercentages1['ExpInc'] = inclassCleanAverages1['iTotalExp']/inclassCleanAverages1['FINCBTXM']
# truncate the max ExpInc
cleanPercentages1.ExpInc.ix[cleanPercentages1['ExpInc']>1] = 1
cleanPercentages1 = cleanPercentages1.ix[:, cleanPercentages1.columns != 'iHousingPrinciple']
cleanPercentages1

In [ ]:
# creating percentage outputs for cleaned dataframe
inclassCleanAverages2 = round(clean2.ix[: ,clean2.columns != 'NEWID'].groupby(['INCLASS'],as_index=False).mean(),2)
# creating new dataframe for the percentages that only includes the plynty categories
cleanPercentages2 = inclassCleanAverages2.loc[:,rollupDict.keys()]
for column in rollupDict.keys():
    cleanPercentages2[column] = inclassCleanAverages2[column]/inclassCleanAverages2.iTotalExp
cleanPercentages2['ExpInc'] = inclassCleanAverages2['iTotalExp']/inclassCleanAverages2['FINCBTXM']
# truncate the max ExpInc
cleanPercentages2nonTruncated = cleanPercentages2.copy()
cleanPercentages2.ExpInc.ix[cleanPercentages2['ExpInc']>1] = 1
cleanPercentages2 = cleanPercentages2.ix[:, cleanPercentages2.columns != 'iHousingPrinciple']
cleanPercentages2

In [ ]:
plt.bar(cleanPercentages2.index, cleanPercentages2.ExpInc, color = 'b')
plt.title("Percent of Income Expended per Income Class (outer fence)")
plt.xlabel("Income Class")
plt.ylabel("Percentage")
plt.show()

In [ ]:
plt.bar(cleanPercentages2.index, cleanPercentages2.iHousing, color = 'b')
plt.title("Percent spent on Housing per Income Class (outer fence)")
plt.xlabel("Income class")
plt.ylabel("Percent spent on housing")
plt.show()

# Plots

In [ ]:
# ploting the number of people in each bracket
print(inclassExpenses.INCLASS.value_counts().values)
plt.bar(list(inclassExpenses.INCLASS.value_counts().index.tolist()), inclassExpenses.INCLASS.value_counts().values, align='center', color = "r")
plt.title("Number of CUs in Income classes (with outliers)")
plt.xlabel("Income Class")
plt.ylabel("Count")
# plt.show()

In [ ]:
# creating the plot of percent of income expended per income class
plt.bar(percentages.index, percentages.ExpInc, color = 'r')
plt.title("Percent of Income Expended per Income Class (with outliers)")
plt.xlabel("Income Class")
plt.ylabel("Percentage")
# plt.show()

In [ ]:
# creating plot of Percent spent on Housing per income class
plt.bar(percentages.index, percentages.iHousing, color = 'r')
plt.ylim(0,.4)
plt.title("Percent spent on Housing per Income Class (with outliers)")
plt.xlabel("Income class")
plt.ylabel("Percent spent on housing")
# plt.show()

In [ ]:
inclassSD = inclassExpenses.groupby(['INCLASS'],as_index=False).std()
inclassSD.iHousing
plt.bar(inclassSD.iHousing.index, inclassSD.iHousing, color = 'r')
plt.title("Standard deviations in Income Classes (with outliers)")
# plt.ylim(0,60000)
plt.xlabel("Income Class")
plt.ylabel("Standard Deviation")
# plt.show()

In [ ]:
# ploting the number of people in each bracket
print(clean1.INCLASS.value_counts().values)
plt.bar(list(clean1.INCLASS.value_counts().index.tolist()), clean1.INCLASS.value_counts().values, align='center', color="g")
plt.title("Number of CUs in Income classes (outer fence)")
plt.xlabel("Income Class")
plt.ylabel("Count")
# plt.show()

In [ ]:
inclassClean1SD = clean1.groupby(['INCLASS'],as_index=False).std()
inclassClean1SD.iHousing
plt.bar(inclassClean1SD.iHousing.index, inclassClean1SD.iHousing,color = 'g')
plt.ylim(0,60000)
plt.title("Standard deviations in Income Classes (inner fence)")
plt.xlabel("Income Class")
plt.ylabel("Standard Deviation")
# plt.show()

In [ ]:
# ploting the number of people in each bracket
print(clean2.INCLASS.value_counts().values)
plt.bar(list(clean2.INCLASS.value_counts().index.tolist()), clean2.INCLASS.value_counts().values, align='center', color="b")
plt.title("Number of CUs in Income classes (outer fence)")
plt.xlabel("Income Class")
plt.ylabel("Count")
# plt.show()

In [ ]:
inclassClean2SD = clean2.groupby(['INCLASS'],as_index=False).std()
inclassClean2SD.iHousing
plt.bar(inclassClean2SD.iHousing.index, inclassClean2SD.iHousing,color = 'b')
plt.ylim(0,60000)
plt.title("Standard deviations in Income Classes (outer fence)")
plt.xlabel("Income Class")
plt.ylabel("Standard Deviation")
# plt.show()

In [ ]:
plt.close()
n_groups = 4
values1 = inclassExpenses.INCLASS.value_counts().values
values2 = clean1.INCLASS.value_counts().values
values3 = clean2.INCLASS.value_counts().values

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(inclassExpenses.INCLASS.value_counts().index -1, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='With')
 
rects2 = plt.bar(clean1.INCLASS.value_counts().index -1 + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='Inner')

rects3 = plt.bar(clean2.INCLASS.value_counts().index - 1 + bar_width*2, values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Outer')

plt.xlabel('Income Class')
plt.ylabel('Number of CUs')
plt.title('Number of CUs per Income Class')
plt.legend()
plt.show()
clean2.INCLASS.value_counts().values

In [ ]:
n_groups = 3
values1 = percentages.iHousing
values2 = cleanPercentages1.iHousing
values3 = cleanPercentages2.iHousing

print("The average percent change between with and without outliers: "+str(round((values1-values3).mean()*100,2))+"%")

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(percentages.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='With')
 
rects2 = plt.bar(cleanPercentages1.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='Inner')

rects3 = plt.bar(cleanPercentages2.index + bar_width*2, values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Outer')

plt.xlabel('Income Class')
plt.ylabel('Percent of Total Expenditures')
plt.title('Percent Spent on Housing per Income Class')
plt.ylim(0,.4)
plt.legend()
plt.show()

In [ ]:
n_groups = 3
values1 = inclassSD.iHousing
values2 = inclassClean1SD.iHousing
values3 = inclassClean2SD.iHousing

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(percentages.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='With')
 
rects2 = plt.bar(cleanPercentages1.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='Inner')

rects3 = plt.bar(cleanPercentages2.index + bar_width*2, values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Outer')

plt.xlabel('Income Class')
plt.ylabel('Standard Deviation (Dollars)')
plt.title('Standard Deviation of money spent on Housing per Income Class')
plt.legend()
plt.show()

In [ ]:
n_groups = 2
values1 = inclassSD.iHousing
values2 = inclassClean2SD.iHousing

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35
opacity = 1
 
rects1 = plt.bar(percentages.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='With')
 
rects2 = plt.bar(cleanPercentages2.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Without')

plt.xlabel('Income Class')
plt.ylabel('Standard Deviation (Dollars)')
plt.suptitle('Standard Deviation of money spent on Housing per Income Class')
plt.title('(With and without outliers)')
plt.legend()
plt.show()

In [ ]:
n_groups = 2
values1 = percentages.iHousing
values2 = cleanPercentages2.iHousing

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35
opacity = 1
 
rects1 = plt.bar(percentages.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='With')
 
rects2 = plt.bar(cleanPercentages2.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Without')

plt.xlabel('Income Class')
plt.ylabel('Percent of Total Expenditures')
plt.suptitle('Percent Expended on Housing per Income Class')
plt.title('(With and without outliers)')
plt.legend()
plt.ylim(0,0.4)
plt.show()

In [ ]:
n_groups = 3
values1 = percentages.ExpInc
values2 = cleanPercentages1.ExpInc
values3 = cleanPercentages2.ExpInc

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(percentages.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='With')
 
rects2 = plt.bar(cleanPercentages1.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='Inner Fence')

rects3 = plt.bar(cleanPercentages2.index + bar_width*2, values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Outer Fence')
 
plt.xlabel('Income Class')
plt.ylabel('Percent of Income Expended')
plt.title('Percent of Income Expent per Income Class')
plt.legend()
plt.show()

# Least squares polynomial fit
any value that would cause a person to expend over 100% of their income is truncated to 100%

In [ ]:
# cleanPercentages2.ExpInc[3:]
cleanReg = clean2[["FINCBTXM","iTotalExp"]]
cleanReg = cleanReg.loc[clean2.FINCBTXM > 0]
cleanReg = cleanReg.loc[cleanReg.iTotalExp > 0]
cleanReg
def getExpendPercent(income):
    if income <= 0:
        return(1)
    coefficients = np.polyfit(cleanReg.FINCBTXM, cleanReg.iTotalExp, deg = 3)
    p = np.poly1d(coefficients)
    np.seterr(divide='ignore')
    percent = p(income)/income
    if percent > 1:
        percent = 1
    return(percent)

def printRegression():
    coefficients = np.polyfit(cleanReg.FINCBTXM, cleanReg.iTotalExp, deg = 3)
    p = np.poly1d(coefficients)
    print(p)
    
    
printRegression()
    
x = range(0,360000,1000)
x2 = range(0,360000,1000)
y =[]
y2 =[]
def oldRegression(income):
    if income > 53000:
        output = ((-7.6108*(10**(-17)))*(income**3))+((4.2009*(10**(-11)))*(income**2))+((-7.90256*(10**-6))*income)+1.21112
    else:
        output = 1
    return(output)


for bracket in range(0,360000,1000):
    y.append(getExpendPercent(bracket))
    y2.append(oldRegression(bracket))
# plt.plot(inclassCleanAverages2.FINCBTXM,cleanPercentages2.ExpInc)
plt.plot(x,y, color = "r", label="Regression")
# plt.plot(x2,y2, color = "b", label = "Old")
x3 = range(0,incomeBrackets[len(incomeBrackets)-2])

# creating the repeatArray for income Bracket plotting
lastvalue = 0
repeatArray = []
for x in range(0,len(incomeBrackets[1:len(incomeBrackets)-1])):
    repeatArray.append(incomeBrackets[1:len(incomeBrackets)-1][x] - lastvalue)
    lastvalue = incomeBrackets[1:len(incomeBrackets)-1][x]

y3 = np.repeat(cleanPercentages2.ExpInc[0:len(cleanPercentages2)-1], repeatArray)
print(x3)
print(len(y3))
plt.plot(x3,y3, color = "g", label = "Brackets")
plt.title("Percent of Income Expended Regression")
plt.xlabel("Income Before Taxes")
plt.ylabel("Percent of Income Expended")
plt.xlim(0,350000)
plt.ylim(0,1.2)
plt.legend()
plt.show()

In [ ]:
plt.scatter(cleanReg.FINCBTXM, cleanReg.iTotalExp, s=1.5)
coefficients = np.polyfit(cleanReg.FINCBTXM, cleanReg.iTotalExp, deg = 3)
p = np.poly1d(coefficients)
np.seterr(divide='ignore')
regX = range(0,600000,1000)
plt.plot(regX, p(regX), color="black")
plt.xlabel("Income before Taxes (Dollars)")
plt.ylabel("Dollars Expent")
plt.title("Regression of Income to Expenditure")
plt.show()

### Plot of number of CUs for clean2 only

In [ ]:
plt.bar(clean2.INCLASS.value_counts().index - 1, (clean2.INCLASS.value_counts().values)/(len(clean2)), alpha=opacity, color='g')
plt.suptitle("Percent of CUs in each Income class")
plt.title("Ages: 60-75")
plt.xlabel("Income Class")
plt.ylabel("Percent of CUs")
plt.show()

# Examples of Users

In [ ]:
usersIncome = [30000, 60000, 90000, 120000, 250000]
def getIncomeBracket(income):
    for bracket in range(len(incomeBrackets)-2,0,-1):
        if income > incomeBrackets[bracket]:
            return bracket
            

exampleExpenses = pd.DataFrame(columns = cleanPercentages2.columns, index = range(len(usersIncome)))
for user in usersIncome:
    userBracket = getIncomeBracket(user)
    userExpendAmount = getExpendPercent(user) * user
    userBaseExpenses = (cleanPercentages2.loc[userBracket]*userExpendAmount)/12
    exampleExpenses.loc[usersIncome.index(user)] = userBaseExpenses

exampleExpenses.index = usersIncome

exampleExpenses = exampleExpenses.ix[:, exampleExpenses.columns != "ExpInc"]
exampleExpenses

# Creating CSV fo percentages

In [ ]:
cleanPercentages2.to_csv("percentage"+str(minAge)+"to"+str(maxAge)+".csv")

# Importing CSVs of pre-Subset Percentages CSVs

In [ ]:
ageRange1 = pd.read_csv("percentage55to64.csv", index_col=0)
ageRange2 = pd.read_csv("percentage60to75.csv", index_col=0)
ageRange3 = pd.read_csv("percentage65to130.csv", index_col=0)

### Plotting the differences between Age brackets

In [ ]:
n_groups = 3
values1 = ageRange1.ExpInc
values2 = ageRange2.ExpInc
values3 = ageRange3.ExpInc

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(ageRange1.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='55-64')
 
rects2 = plt.bar(ageRange2.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='60-75')

rects3 = plt.bar(ageRange3.index + (bar_width * 2), values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='65+')

plt.xlabel('Income Class')
plt.ylabel('Percent of Income Expended')
plt.title('Percent of Income Expent per Income Class')
plt.legend()
plt.show()

In [ ]:
n_groups = 3
values1 = ageRange1.iHousing
values2 = ageRange2.iHousing
values3 = ageRange3.iHousing

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(ageRange1.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='55-64')
 
rects2 = plt.bar(ageRange2.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='60-75')

rects3 = plt.bar(ageRange3.index + (bar_width * 2), values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='65+')

plt.xlabel('Income Class')
plt.ylabel('Percent of Total Expenditures')
plt.title('Percent Spent on Housing per Income Class')
plt.legend()
plt.ylim(0,.4)
plt.show()

In [ ]:
n_groups = 3
values1 = ageRange1.iHealthcare
values2 = ageRange2.iHealthcare
values3 = ageRange3.iHealthcare

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(ageRange1.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='55-64')
 
rects2 = plt.bar(ageRange2.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='60-75')

rects3 = plt.bar(ageRange3.index + (bar_width * 2), values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='65+')

plt.xlabel('Income Class')
plt.ylabel('Percent of Total Expenditures')
plt.title('Percent Spent on Healthcare per Income Class')
plt.legend()
plt.show()

In [ ]:
n_groups = 2
values1 = ageRange1.iTransportation
values2 = ageRange2.iTransportation
values3 = ageRange3.iTransportation

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.25
opacity = 1
 
rects1 = plt.bar(ageRange1.index, values1, bar_width,
                 alpha=opacity,
                 color='r',
                 label='55-64')
 
rects2 = plt.bar(ageRange2.index + bar_width, values2, bar_width,
                 alpha=opacity,
                 color='g',
                 label='60-75')

rects3 = plt.bar(ageRange3.index + (bar_width * 2), values3, bar_width,
                 alpha=opacity,
                 color='b',
                 label='65+')

plt.xlabel('Income Class')
plt.ylabel('Percent of Total Expenditures')
plt.title('Percent Spent on Transportation per Income Class')
plt.legend()
plt.show()

# Thoughts on the difference between 55-64, 60-75, and 65+
- On average: 55-64 < 60-75 < 65+ in terms of expenditure percentage

## What each bracket represents
- 55-64 simulates that specific time right before and right at the begining of retirment
- 60-75 simulates the time during retirement with some non-retired CUs
- 65+ is a higher potential to show people who have actually retired